In [1]:
!pip install -qq torch torchvision trimesh scipy open3d plotly

In [2]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import trimesh
from scipy.ndimage import zoom
import open3d as o3d
import plotly.graph_objects as go
import matplotlib.pyplot as plt

# Check if CUDA is available, else use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
!rm -rf ModelNet10.zip ModelNet10
!wget http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip
!unzip -q ModelNet10.zip

--2024-08-21 16:08:41--  http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip
Resolving 3dvision.princeton.edu (3dvision.princeton.edu)... 128.112.136.67
Connecting to 3dvision.princeton.edu (3dvision.princeton.edu)|128.112.136.67|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip [following]
--2024-08-21 16:08:41--  https://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip
Connecting to 3dvision.princeton.edu (3dvision.princeton.edu)|128.112.136.67|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 473402300 (451M) [application/zip]
Saving to: ‘ModelNet10.zip’

ModelNet10.zip      100%[===================>] 451.47M  89.6MB/s    in 4.6s    

2024-08-21 16:08:46 (99.0 MB/s) - ‘ModelNet10.zip’ saved [473402300/473402300]



In [5]:
test_path = "ModelNet10/bed/train/bed_0001.off"
mesh = o3d.io.read_triangle_mesh(test_path)
def mesh_to_plotly(mesh):
    # Extract vertices and faces from the mesh
    vertices = np.asarray(mesh.vertices)
    triangles = np.asarray(mesh.triangles)

    # Create the plotly mesh
    fig = go.Figure(data=[go.Mesh3d(
        x=vertices[:, 0],
        y=vertices[:, 1],
        z=vertices[:, 2],
        i=triangles[:, 0],
        j=triangles[:, 1],
        k=triangles[:, 2],
        opacity=0.5,
        color='lightblue'
    )])

    fig.update_layout(
        scene=dict(
            xaxis=dict(title='X'),
            yaxis=dict(title='Y'),
            zaxis=dict(title='Z')
        ),
        title='3D Mesh Visualization'
    )

    fig.show()

# mesh_to_plotly(mesh)

In [ ]:
def plot_voxel_grid(mesh_path):
    mesh = o3d.io.read_triangle_mesh(mesh_path)
    voxel = o3d.geometry.VoxelGrid.create_from_triangle_mesh(mesh, 0.05)
    # Extract voxel grid data
    voxels = np.array([v.grid_index for v in voxel.get_voxels()])

    # Determine voxel grid size
    voxel_size = voxel.voxel_size
    x_size = int(np.max(voxels[:, 0]) + 1)
    y_size = int(np.max(voxels[:, 1]) + 1)
    z_size = int(np.max(voxels[:, 2]) + 1)

    # Create an empty 3D array
    grid = np.zeros((x_size, y_size, z_size))

    # Populate the grid
    for voxel_index in voxels:
        grid[voxel_index[0], voxel_index[1], voxel_index[2]] = 1

    # Plot slices of the voxel grid
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))

    slices = [grid[:, :, z_size // 2], grid[:, y_size // 2, :], grid[x_size // 2, :, :]]
    titles = ['XY Plane', 'XZ Plane', 'YZ Plane']

    for ax, slice_, title in zip(axes, slices, titles):
        cax = ax.imshow(slice_, cmap='gray', origin='lower')
        ax.set_title(title)
        ax.axis('off')

    plt.colorbar(cax, ax=axes.tolist())
    plt.show()

# Example usage
mesh_path = 'ModelNet10/bed/train/bed_0001.off'
plot_voxel_grid(mesh_path)

In [16]:
class ModelNet10Dataset(Dataset):
    def __init__(self, root_dir, transform=None, train=True):
        self.root_dir = root_dir
        self.transform = transform
        self.data = []
        self.labels = []
        # Filter only directories for class names
        self.classes = [d for d in sorted(os.listdir(root_dir)) if os.path.isdir(os.path.join(root_dir, d))]
        subfolder = 'train' if train else 'test'
        for idx, label in enumerate(self.classes):
            new_path = os.path.join(root_dir, label, subfolder)
            if os.path.isdir(new_path):  # Ensure this is a directory
                for file in os.listdir(new_path):
                    file_path = os.path.join(new_path, file)
                    if os.path.isfile(file_path) and file_path.endswith('.off'):  # Ensure this is a .off file
                        self.data.append(file_path)
                        self.labels.append(idx)

    def __len__(self):
        return len(self.data)

    @staticmethod
    def get_voxel_grid(mesh_path):
        mesh = o3d.io.read_triangle_mesh(mesh_path)
        voxel = o3d.geometry.VoxelGrid.create_from_triangle_mesh(mesh, 32)
        return voxel

    @staticmethod
    def get_np_array(voxel_grid):
        arr = np.zeros((32, 32, 32), dtype=np.uint8)
        for voxel in voxel_grid.get_voxels():
            coord = voxel.grid_index
            arr[coord[0], coord[1], coord[2]] = 1
        return arr

    def __getitem__(self, idx):
        voxel_grid = ModelNet10Dataset.get_voxel_grid(self.data[idx])
        arr = ModelNet10Dataset.get_np_array(voxel_grid)
        print(arr)
        # voxel_grid = torch.tensor(arr, dtype=torch.float32)
        if self.transform:
            voxel_grid = self.transform(voxel_grid)
        label = self.labels[idx]
        return voxel_grid, label

In [17]:
dataset = ModelNet10Dataset(root_dir='ModelNet10', train=True)

In [18]:
dataset[0]

[[[1 1 0 ... 0 0 0]
  [1 1 0 ... 0 0 0]
  [1 1 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[1 1 0 ... 0 0 0]
  [1 1 0 ... 0 0 0]
  [1 1 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 ...

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]]


(VoxelGrid with 12 voxels., 0)

In [ ]:
class VoxNet(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        self.n_classes = n_classes
        self.net = nn.Sequential(
            nn.Conv3d(1, 32, kernel_size=5, stride=2),
            nn.ReLU(),
            # nn.MaxPool3d(2),
            nn.Conv3d(32, 32, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.MaxPool3d(2), # max pool?
            nn.Flatten(),
            nn.Linear(32 * 6 * 6 * 6, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, n_classes)
        )

    def forward(self, x):
        if x.dim() == 3:
            x = x.unsqueeze(0)
        return self.net(x)

model = VoxNet().to(device)

In [ ]:
def train_model(model, train_loader, optimizer, criterion, device):
    model.train()
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

def evaluate_model(model, test_loader, device):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    return correct / len(test_loader.dataset)

# Hyperparameters
epochs = 10
lr = 0.001
batch_size = 32

# Set up data loaders
train_dataset = ModelNet10Dataset(root_dir='ModelNet10', train=True)
test_dataset = ModelNet10Dataset(root_dir='ModelNet10', train=False)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Set up the optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

# Training loop
for epoch in range(epochs):
    train_model(model, train_loader, optimizer, criterion, device)
    accuracy = evaluate_model(model, test_loader, device)
    print(f'Epoch {epoch+1}, Accuracy: {accuracy:.2f}')

Epoch 1, Accuracy: 0.71
